<a href="https://colab.research.google.com/github/sgrigerc/aggregation_df/blob/main/%D0%B4%D0%B71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
#Загружаем датасеты
user_table = pd.read_csv('user_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')
home_page = pd.read_csv('home_page_table.csv')



In [ ]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation_page'})


In [ ]:
# Склеиваем все датасеты все сразу через синтаксический сахар
df = user_table.merge(home_page, on = 'user_id', how = 'left')\
.merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left')\
.merge(payment_confirmation, on = 'user_id', how = 'left')

df.head()

In [ ]:
df['device'].unique()

array(['Desktop', 'Mobile'], dtype=object)

In [ ]:
#Перекодируем названия событий в 0/1

df['payment_confirmation_page'] = df['payment_confirmation_page'].fillna(0)
condition = df['payment_confirmation_page'] == 'payment_confirmation_page'
df.loc[condition, 'payment_confirmation_page'] = 1
df.head()

df['payment_page'] = df['payment_page'].fillna(0)
condition = df['payment_page'] == 'payment_page'
df.loc[condition, 'payment_page'] = 1
df.head()

df['search_page'] = df['search_page'].fillna(0)
condition = df['search_page'] == 'search_page'
df.loc[condition, 'search_page'] = 1
df.head()

df['home_page'] = df['home_page'].fillna(0)
condition = df['home_page'] == 'home_page'
df.loc[condition, 'home_page'] = 1
df.head()


,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [ ]:
pd.pivot_table(df, index = 'sex', columns = 'device', values = 'home_page', aggfunc = 'sum', margins=True)


device,Desktop,Mobile,All
sex,,,
Female,29997,15078,45075
Male,30203,15122,45325
All,60200,30200,90400


#П1-построение сегментированной воронки конверсии пользователей интернет-магазина по полу


In [ ]:

# Импортируем графическую библиотеку Plotly
from plotly import graph_objects as go

#Используем функцию crosstab
# (по умолчанию рассчитывает частоты, поэтому valus = df['user_id'] и aggfunc = 'count' можно не указывать)
df.groupby(['sex'])['user_id'].count()

dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation_page']].sum().unstack('sex').unstack('sex').reset_index()

# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None

# Переименуем стандартные после процедуры заголовки
dfg = dfg.rename(columns = {'index':'Page'})

# dfg.head()

# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

# # Часть графика для типа полу
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Female'],
    name = 'Female',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

# Часть графика для типа устройств Mobile
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Male'],
    name = 'Male',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()



In [ ]:
user_table = pd.read_csv('user_table.csv')
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [ ]:
df = pd.read_csv('user_table.csv')

import datetime

#месяц
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
df['month'] = df['date'].dt.strftime('%m')

#день недели
df['day_of_week'] = df['date'].dt.weekday

df['DayWeek'] = df['date']. dt.day_name ()

# df.head()

# summary = df[['day_of_week', 'payment_confirmation_page']].sum().reset_index()
# summary.head()

df.groupby('day_of_week', ).sum(1).reset_index()







,day_of_week,user_id
0,0,6539406842
1,1,6308086374
2,2,6372183346
3,3,6725390763
4,4,6370653718
5,5,6375129612
6,6,6392504731


In [ ]:
#П3 - определить самый топовый день по продажам месяц и день недели sum или count

df = pd.read_csv('user_table.csv')\
.merge(payment_confirmation, on = 'user_id', how = 'left')

df['payment_confirmation_page'] = df['payment_confirmation_page'].fillna(0)
condition = df['payment_confirmation_page'] == 'payment_confirmation_page'
df.loc[condition, 'payment_confirmation_page'] = 1

import datetime

#месяц
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
df['month'] = df['date'].dt.strftime('%m')

#день недели
df['day'] = df['date'].dt.weekday

df_marks = df.drop(['user_id', 'date', 'device', 'sex'], axis=1)

# Агрегируем данные: подсчитываем количество продаж на каждый день недели
pd.pivot_table(df, index = 'day', values = 'payment_confirmation_page',  aggfunc = 'sum', margins=True)

# Агрегируем данные: подсчитываем количество продаж на каждый день недели
pd.pivot_table(df, index = 'month', values = 'payment_confirmation_page',  aggfunc = 'sum', margins=True)

#Определение максимальных значений


,payment_confirmation_page
month,
01,189
02,173
03,44
04,46
All,452
